# All oscillation analysis

Brief 1-2 sentence description of notebook.

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import os
import collections
import itertools
from collections import defaultdict
from itertools import combinations

In [3]:
os.environ["SPECTRAL_CONNECTIVITY_ENABLE_GPU"] = "true"
import cupy as cp

In [4]:
# Imports of all used packages and libraries
import numpy as np
import pandas as pd
from scipy import stats
from scipy.stats import mannwhitneyu


In [5]:
import matplotlib
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns
import colorsys

In [6]:
from sklearn.metrics import confusion_matrix

In [7]:
from spectral_connectivity import Multitaper, Connectivity
import spectral_connectivity

In [8]:
FONTSIZE = 20

In [9]:
font = {'weight' : 'medium',
        'size'   : 20}

matplotlib.rc('font', **font)

# Functions

In [10]:
def generate_pairs(lst):
    """
    Generates all unique pairs from a list.

    Parameters:
    - lst (list): The list to generate pairs from.

    Returns:
    - list: A list of tuples, each containing a unique pair from the input list.
    """
    n = len(lst)
    return [(lst[i], lst[j]) for i in range(n) for j in range(i+1, n)]

## Inputs & Data

Explanation of each input and where it comes from.

In [11]:
# Inputs and Required data loading
# input varaible names are in all caps snake case
# Whenever an input changes or is used for processing 
# the vairables are all lower in snake case
OUTPUT_DIR = r"./proc" # where data is saved should always be shown in the inputs

In [12]:
TIME_HALFBANDWIDTH_PRODUCT = 2
TIME_WINDOW_DURATION = 1
TIME_WINDOW_STEP = 0.5
RESAMPLE_RATE=1000

In [13]:
BAND_TO_FREQ = {"theta": (4,12), "gamma": (30,51)}

In [14]:
LFP_TRACES_DF = pd.read_pickle("./proc/rce_pilot_2_01_standard_lfp_traces_and_frames.pkl")

In [15]:
LFP_TRACES_DF.head()

,cohort,session_dir,tone_frames,box_1_port_entry_frames,box_2_port_entry_frames,video_name,session_path,recording,current_subject,subject,...,video_timestamps,tone_timestamps,box_1_port_entry_timestamps,box_2_port_entry_timestamps,lfp_timestamps,mPFC_lfp_trace,MD_lfp_trace,LH_lfp_trace,BLA_lfp_trace,vHPC_lfp_trace
0,2,20230612_101430_standard_comp_to_training_D1_s...,"[[980, 1181], [3376, 3575], [5672, 5871], [746...","[[490, 514], [518, 558], [558, 637], [638, 640...","[[33137, 33147], [33665, 33666], [33668, 33669...",20230612_101430_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_101430_standard_comp_to_training_D1_s...,1.3,1.3,...,"[-2, 1384, 2770, 4156, 4156, 5542, 6928, 6928,...","[[982229, 1182226], [3382227, 3582224], [56822...","[[491029, 515227], [519426, 558629], [559427, ...","[[33082200, 33090003], [33565003, 33567000], [...","[0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 2...","[494, 424, 501, 678, 634, 474, 384, 461, 570, ...","[231, 247, 378, 494, 389, 200, 109, 107, 49, -...","[307, 330, 409, 453, 356, 197, 124, 126, 83, 3...","[292, 290, 384, 450, 311, 111, -4, -9, -66, -2...","[287, 407, 656, 872, 969, 959, 936, 957, 985, ..."
1,2,20230612_101430_standard_comp_to_training_D1_s...,"[[980, 1180], [3376, 3575], [5672, 5871], [746...","[[490, 514], [518, 558], [558, 637], [638, 640...","[[33021, 33027], [33502, 33503], [33504, 33506...",20230612_101430_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_101430_standard_comp_to_training_D1_s...,1.3,1.3,...,"[-2, 1384, 2770, 4156, 4156, 5542, 6928, 6928,...","[[982229, 1182226], [3382227, 3582224], [56822...","[[491029, 515227], [519426, 558629], [559427, ...","[[33082200, 33090003], [33565003, 33567000], [...","[0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 2...","[494, 424, 501, 678, 634, 474, 384, 461, 570, ...","[231, 247, 378, 494, 389, 200, 109, 107, 49, -...","[307, 330, 409, 453, 356, 197, 124, 126, 83, 3...","[292, 290, 384, 450, 311, 111, -4, -9, -66, -2...","[287, 407, 656, 872, 969, 959, 936, 957, 985, ..."
2,2,20230612_101430_standard_comp_to_training_D1_s...,"[[980, 1181], [3376, 3575], [5672, 5871], [746...","[[490, 514], [518, 558], [558, 637], [638, 640...","[[33137, 33147], [33665, 33666], [33668, 33669...",20230612_101430_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_101430_standard_comp_to_training_D1_s...,1.4,1.4,...,"[-2, 1384, 2770, 4156, 4156, 5542, 6928, 6928,...","[[982229, 1182226], [3382227, 3582224], [56822...","[[491029, 515227], [519426, 558629], [559427, ...","[[33082200, 33090003], [33565003, 33567000], [...","[0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 2...","[95, 144, 164, 188, 227, 84, -179, -384, -466,...","[147, 144, 129, 115, 105, 59, 0, -13, -29, -98...","[142, 126, 109, 113, 114, 68, 0, 0, -1, -100, ...","[327, 438, 456, 342, 437, 615, 647, 649, 678, ...","[308, 430, 520, 431, 373, 450, 552, 518, 429, ..."
3,2,20230612_101430_standard_comp_to_training_D1_s...,"[[980, 1180], [3376, 3575], [5672, 5871], [746...","[[490, 514], [518, 558], [558, 637], [638, 640...","[[33021, 33027], [33502, 33503], [33504, 33506...",20230612_101430_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_101430_standard_comp_to_training_D1_s...,1.4,1.4,...,"[-2, 1384, 2770, 4156, 4156, 5542, 6928, 6928,...","[[982229, 1182226], [3382227, 3582224], [56822...","[[491029, 515227], [519426, 558629], [559427, ...","[[33082200, 33090003], [33565003, 33567000], [...","[0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 2...","[95, 144, 164, 188, 227, 84, -179, -384, -466,...","[147, 144, 129, 115, 105, 59, 0, -13, -29, -98...","[142, 126, 109, 113, 114, 68, 0, 0, -1, -100, ...","[327, 438, 456, 342, 437, 615, 647, 649, 678, ...","[308, 430, 520, 431, 373, 450, 552, 518, 429, ..."
4,2,20230612_112630_standard_comp_to_training_D1_s...,"[[1125, 1324], [3519, 3720], [5815, 6014], [76...","[[192, 24

In [16]:
LFP_TRACES_DF.shape

(61, 23)

# Power Calcuation

- Getting the column name of all the traces

In [17]:
trace_columns = [col for col in LFP_TRACES_DF.columns if "lfp_trace" in col]

In [18]:
trace_columns

['mPFC_lfp_trace',
 'MD_lfp_trace',
 'LH_lfp_trace',
 'BLA_lfp_trace',
 'vHPC_lfp_trace']

In [19]:
for col in trace_columns:
    print(col)
    LFP_TRACES_DF[col] = LFP_TRACES_DF[col].apply(lambda x: x.astype(np.float16))

mPFC_lfp_trace
MD_lfp_trace
LH_lfp_trace
BLA_lfp_trace
vHPC_lfp_trace


- Calcuating the power at each frequency band

In [20]:
for col in trace_columns:
    brain_region = col.split("_")[0]
    print(brain_region)

    # Define column names
    lfp_trace_col = f"{brain_region}_lfp_trace"
    multitaper_col = f"{brain_region}_power_multitaper"
    connectivity_col = f"{brain_region}_power_connectivity"
    frequencies_col = f"{brain_region}_power_frequencies"
    power_col = f"{brain_region}_power_all_frequencies_all_windows"
    
    try:
        # Apply Multitaper function to the lfp_trace column
        LFP_TRACES_DF[multitaper_col] = LFP_TRACES_DF[lfp_trace_col].apply(
            lambda x: Multitaper(
                time_series=x, 
                sampling_frequency=RESAMPLE_RATE, 
                time_halfbandwidth_product=TIME_HALFBANDWIDTH_PRODUCT,
                time_window_duration=TIME_WINDOW_DURATION, 
                time_window_step=TIME_WINDOW_STEP
            )
        )

        # Apply Connectivity function to the multitaper column
        LFP_TRACES_DF[connectivity_col] = LFP_TRACES_DF[multitaper_col].apply(
            lambda x: Connectivity.from_multitaper(x)
        )

        # Apply frequencies and power functions to the connectivity column
        LFP_TRACES_DF[frequencies_col] = LFP_TRACES_DF[connectivity_col].apply(
            lambda x: x.frequencies
        )
        LFP_TRACES_DF[power_col] = LFP_TRACES_DF[connectivity_col].apply(
            lambda x: x.power().squeeze()
        )
        
        LFP_TRACES_DF[power_col] = LFP_TRACES_DF[power_col].apply(lambda x: x.astype(np.float16))
            
        # Removing unnecessary columns
        LFP_TRACES_DF = LFP_TRACES_DF.drop(columns=[multitaper_col, connectivity_col], errors="ignore")
    
    except Exception as e: 
        print(e)

mPFC
MD
LH
BLA
vHPC


- Getting the timestamps of the power

In [21]:
LFP_TRACES_DF["power_timestamps"] = LFP_TRACES_DF["lfp_timestamps"].apply(lambda x: x[(RESAMPLE_RATE//2):(-RESAMPLE_RATE//2):(RESAMPLE_RATE//2)])
# .iloc[0][500:-500:500].shape

- Making sure that the timestamps for power makes sense with shape and values

In [22]:
LFP_TRACES_DF["power_timestamps"].head().apply(lambda x: x.shape)

0    (6828,)
1    (6828,)
2    (6828,)
3    (6828,)
4    (6833,)
Name: power_timestamps, dtype: object

In [23]:
LFP_TRACES_DF[[col for col in LFP_TRACES_DF.columns if "power_all_frequencies_all_windows" in col][0]].iloc[0].shape

(6828, 500)

In [24]:
LFP_TRACES_DF[[col for col in LFP_TRACES_DF.columns if "lfp_timestamps" in col][0]].iloc[0]

array([       0,       20,       40, ..., 68293300, 68293320, 68293340],
      dtype=int32)

In [25]:
LFP_TRACES_DF[[col for col in LFP_TRACES_DF.columns if "power_timestamps" in col][0]].iloc[0]

array([   10000,    20000,    30000, ..., 68260000, 68270000, 68280000],
      dtype=int32)

- Checking if the right frequencies are being used

In [26]:
LFP_TRACES_DF[[col for col in LFP_TRACES_DF.columns if "power_frequencies" in col]].head()

,mPFC_power_frequencies,MD_power_frequencies,LH_power_frequencies,BLA_power_frequencies,vHPC_power_frequencies
0,"[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ..."
1,"[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ..."
2,"[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ..."
3,"[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ..."
4,"[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ..."


In [27]:
LFP_TRACES_DF["power_calculation_frequencies"] = LFP_TRACES_DF[[col for col in LFP_TRACES_DF.columns if "power_frequencies" in col][0]].copy()

- Dropping unnecessary columns

In [28]:
LFP_TRACES_DF = LFP_TRACES_DF.drop(columns=[col for col in LFP_TRACES_DF.columns if "power_frequencies" in col], errors="ignore")

In [29]:
LFP_TRACES_DF.head()

,cohort,session_dir,tone_frames,box_1_port_entry_frames,box_2_port_entry_frames,video_name,session_path,recording,current_subject,subject,...,LH_lfp_trace,BLA_lfp_trace,vHPC_lfp_trace,mPFC_power_all_frequencies_all_windows,MD_power_all_frequencies_all_windows,LH_power_all_frequencies_all_windows,BLA_power_all_frequencies_all_windows,vHPC_power_all_frequencies_all_windows,power_timestamps,power_calculation_frequencies
0,2,20230612_101430_standard_comp_to_training_D1_s...,"[[980, 1181], [3376, 3575], [5672, 5871], [746...","[[490, 514], [518, 558], [558, 637], [638, 640...","[[33137, 33147], [33665, 33666], [33668, 33669...",20230612_101430_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_101430_standard_comp_to_training_D1_s...,1.3,1.3,...,"[307.0, 330.0, 409.0, 453.0, 356.0, 197.0, 124...","[292.0, 290.0, 384.0, 450.0, 311.0, 111.0, -4....","[287.0, 407.0, 656.0, 872.0, 969.0, 959.0, 936...","[[12550.0, 24160.0, 27040.0, 21470.0, 11690.0,...","[[1570.0, 4332.0, 5268.0, 6040.0, 4020.0, 3758...","[[2278.0, 5650.0, 6710.0, 6348.0, 4022.0, 2040...","[[7340.0, 4736.0, 8950.0, 9464.0, 10370.0, 690...","[[12500.0, 7440.0, 21180.0, 20160.0, 25630.0, ...","[10000, 20000, 30000, 40000, 50000, 60000, 700...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ..."
1,2,20230612_101430_standard_comp_to_training_D1_s...,"[[980, 1180], [3376, 3575], [5672, 5871], [746...","[[490, 514], [518, 558], [558, 637], [638, 640...","[[33021, 33027], [33502, 33503], [33504, 33506...",20230612_101430_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_101430_standard_comp_to_training_D1_s...,1.3,1.3,...,"[307.0, 330.0, 409.0, 453.0, 356.0, 197.0, 124...","[292.0, 290.0, 384.0, 450.0, 311.0, 111.0, -4....","[287.0, 407.0, 656.0, 872.0, 969.0, 959.0, 936...","[[12550.0, 24160.0, 27040.0, 21470.0, 11690.0,...","[[1570.0, 4332.0, 5268.0, 6040.0, 4020.0, 3758...","[[2278.0, 5650.0, 6710.0, 6348.0, 4022.0, 2040...","[[7340.0, 4736.0, 8950.0, 9464.0, 10370.0, 690...","[[12500.0, 7440.0, 21180.0, 20160.0, 25630.0, ...","[10000, 20000, 30000, 40000, 50000, 60000, 700...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ..."
2,2,20230612_101430_standard_comp_to_training_D1_s...,"[[980, 1181], [3376, 3575], [5672, 5871], [746...","[[490, 514], [518, 558], [558, 637], [638, 640...","[[33137, 33147], [33665, 33666], [33668, 33669...",20230612_101430_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_101430_standard_comp_to_training_D1_s...,1.4,1.4,...,"[142.0, 126.0, 109.0, 113.0, 114.0, 68.0, 0.0,...","[327.0, 438.0, 456.0, 342.0, 437.0, 615.0, 647...","[308.0, 430.0, 520.0, 431.0, 373.0, 450.0, 552...","[[34.4, 79.2, 288.8, 397.2, 822.5, 812.0, 2098...","[[393.5, 410.0, 349.8, 123.2, 91.5, 39.62, 721...","[[502.5, 540.5, 491.2, 184.4, 130.4, 66.2, 739...","[[80.8, 362.8, 1082.0, 1243.0, 1238.0, 235.2, ...","[[155.5, 233.6, 718.5, 793.0, 795.0, 787.0, 46...","[10000, 20000, 30000, 40000, 50000, 60000, 700...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ..."
3,2,20230612_101430_standard_comp_to_training_D1_s...,"[[980, 1180], [3376, 3575], [5672, 5871], [746...","[[490, 514], [518, 558], [558, 637], [638, 640...","[[33021, 33027], [33502, 33503], [33504, 33506...",20230612_101430_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_101430_standard_comp_to_training_D1_s...,1.4,1.4,...,"[142.0, 126.0, 109.0, 113.0, 114.0, 68.0, 0.0,...","[327.0, 438.0, 456.0, 342.0, 437.0, 615.0, 647...","[308.0, 430.0, 520.0, 431.0, 373.0, 450.0, 552...","[[34.4, 79.2, 288.8, 397.2, 822.5, 812.0, 2098...","[[393.5, 410.0, 349.8, 123.2, 91.5, 39.62, 721...","[[502.5, 540.5, 491.2, 184.4, 130.4, 66.2, 739...","[[80.8, 362.8, 1082.0, 1243.0, 1238.0, 235.2, ...","[[155.5, 233.6, 718.5, 793.0, 795.0, 787.0, 46...","[10000, 20000, 30000, 40000, 50000, 60000, 700...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ..."
4,2,2023061

## Coherence Calculation

- Getting the trace column pairs

In [30]:
trace_columns

['mPFC_lfp_trace',
 'MD_lfp_trace',
 'LH_lfp_trace',
 'BLA_lfp_trace',
 'vHPC_lfp_trace']

In [31]:
brain_region_pairs = generate_pairs(sorted(trace_columns))
brain_region_pairs = sorted(brain_region_pairs)


In [32]:
brain_region_pairs

[('BLA_lfp_trace', 'LH_lfp_trace'),
 ('BLA_lfp_trace', 'MD_lfp_trace'),
 ('BLA_lfp_trace', 'mPFC_lfp_trace'),
 ('BLA_lfp_trace', 'vHPC_lfp_trace'),
 ('LH_lfp_trace', 'MD_lfp_trace'),
 ('LH_lfp_trace', 'mPFC_lfp_trace'),
 ('LH_lfp_trace', 'vHPC_lfp_trace'),
 ('MD_lfp_trace', 'mPFC_lfp_trace'),
 ('MD_lfp_trace', 'vHPC_lfp_trace'),
 ('mPFC_lfp_trace', 'vHPC_lfp_trace')]

## Coherece Calculation

- Calculating the coherence

In [33]:
for region_1, region_2 in brain_region_pairs:
    # Define base name for pair
    pair_base_name = f"{region_1.split('_')[0]}_{region_2.split('_')[0]}"
    print(pair_base_name)

    try:
        # Define column names
        multitaper_col = f"{pair_base_name}_coherence_multitaper"
        connectivity_col = f"{pair_base_name}_coherence_connectivity"
        frequencies_col = f"{pair_base_name}_coherence_frequencies"
        coherence_col = f"{pair_base_name}_coherence_all_frequencies_all_windows"

        # Apply Multitaper function
        LFP_TRACES_DF[multitaper_col] = LFP_TRACES_DF.apply(
            lambda x: Multitaper(
                time_series=np.array([x[region_1], x[region_2]]).T, 
                sampling_frequency=RESAMPLE_RATE, 
                time_halfbandwidth_product=TIME_HALFBANDWIDTH_PRODUCT, 
                time_window_step=TIME_WINDOW_STEP, 
                time_window_duration=TIME_WINDOW_DURATION
            ), 
            axis=1
        )

        # Apply Connectivity function
        LFP_TRACES_DF[connectivity_col] = LFP_TRACES_DF[multitaper_col].apply(
            lambda x: Connectivity.from_multitaper(x)
        )

        # Apply frequencies and coherence functions
        LFP_TRACES_DF[frequencies_col] = LFP_TRACES_DF[connectivity_col].apply(
            lambda x: x.frequencies
        )
        LFP_TRACES_DF[coherence_col] = LFP_TRACES_DF[connectivity_col].apply(
            lambda x: x.coherence_magnitude()[:,:,0,1]
        )

        LFP_TRACES_DF[coherence_col] = LFP_TRACES_DF[coherence_col].apply(lambda x: x.astype(np.float16))

    except Exception as e: 
        print(e)

    # Drop temporary columns
    LFP_TRACES_DF = LFP_TRACES_DF.drop(columns=[multitaper_col, connectivity_col], errors="ignore")

BLA_LH
BLA_MD
BLA_mPFC
BLA_vHPC
LH_MD
LH_mPFC
LH_vHPC
MD_mPFC
MD_vHPC
mPFC_vHPC


- Getting the timestamps of the coherence

In [34]:
LFP_TRACES_DF["coherence_timestamps"] = LFP_TRACES_DF["lfp_timestamps"].apply(lambda x: x[(RESAMPLE_RATE//2):(-RESAMPLE_RATE//2):(RESAMPLE_RATE//2)])


- Making sure that the timestamps for coherence makes sense with shape and values

In [35]:
LFP_TRACES_DF["coherence_timestamps"].head().apply(lambda x: x.shape)

0    (6828,)
1    (6828,)
2    (6828,)
3    (6828,)
4    (6833,)
Name: coherence_timestamps, dtype: object

In [36]:
LFP_TRACES_DF[[col for col in LFP_TRACES_DF.columns if "coherence_all_frequencies_all_windows" in col][0]].iloc[0].shape

(6828, 500)

In [37]:
LFP_TRACES_DF[[col for col in LFP_TRACES_DF.columns if "lfp_timestamps" in col][0]].iloc[0]

array([       0,       20,       40, ..., 68293300, 68293320, 68293340],
      dtype=int32)

In [38]:
LFP_TRACES_DF[[col for col in LFP_TRACES_DF.columns if "coherence_timestamps" in col][0]].iloc[0]

array([   10000,    20000,    30000, ..., 68260000, 68270000, 68280000],
      dtype=int32)

- Checking if the right frequencies are being used

In [39]:
LFP_TRACES_DF[[col for col in LFP_TRACES_DF.columns if "coherence_frequencies" in col]].head()

,BLA_LH_coherence_frequencies,BLA_MD_coherence_frequencies,BLA_mPFC_coherence_frequencies,BLA_vHPC_coherence_frequencies,LH_MD_coherence_frequencies,LH_mPFC_coherence_frequencies,LH_vHPC_coherence_frequencies,MD_mPFC_coherence_frequencies,MD_vHPC_coherence_frequencies,mPFC_vHPC_coherence_frequencies
0,"[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ..."
1,"[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ..."
2,"[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ..."
3,"[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ..."
4,"[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ..."


In [40]:
LFP_TRACES_DF["coherence_calculation_frequencies"] = LFP_TRACES_DF[[col for col in LFP_TRACES_DF.columns if "coherence_frequencies" in col][0]].copy()

- Dropping unnecessary columns

In [41]:
LFP_TRACES_DF = LFP_TRACES_DF.drop(columns=[col for col in LFP_TRACES_DF.columns if "coherence_frequencies" in col], errors="ignore")

In [42]:
LFP_TRACES_DF.head()

,cohort,session_dir,tone_frames,box_1_port_entry_frames,box_2_port_entry_frames,video_name,session_path,recording,current_subject,subject,...,BLA_mPFC_coherence_all_frequencies_all_windows,BLA_vHPC_coherence_all_frequencies_all_windows,LH_MD_coherence_all_frequencies_all_windows,LH_mPFC_coherence_all_frequencies_all_windows,LH_vHPC_coherence_all_frequencies_all_windows,MD_mPFC_coherence_all_frequencies_all_windows,MD_vHPC_coherence_all_frequencies_all_windows,mPFC_vHPC_coherence_all_frequencies_all_windows,coherence_timestamps,coherence_calculation_frequencies
0,2,20230612_101430_standard_comp_to_training_D1_s...,"[[980, 1181], [3376, 3575], [5672, 5871], [746...","[[490, 514], [518, 558], [558, 637], [638, 640...","[[33137, 33147], [33665, 33666], [33668, 33669...",20230612_101430_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_101430_standard_comp_to_training_D1_s...,1.3,1.3,...,"[[0.888, 0.7446, 0.7627, 0.573, 0.5938, 0.2527...","[[0.8643, 0.9487, 0.8594, 0.887, 0.2457, 0.050...","[[0.8984, 0.9756, 0.98, 0.966, 0.903, 0.9233, ...","[[0.985, 0.9976, 0.995, 0.9863, 0.991, 0.7124,...","[[0.5127, 0.671, 0.6772, 0.729, 0.4365, 0.2106...","[[0.96, 0.9683, 0.9746, 0.949, 0.8516, 0.716, ...","[[0.812, 0.632, 0.6074, 0.619, 0.2316, 0.2827,...","[[0.6343, 0.714, 0.6826, 0.702, 0.477, 0.4604,...","[10000, 20000, 30000, 40000, 50000, 60000, 700...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ..."
1,2,20230612_101430_standard_comp_to_training_D1_s...,"[[980, 1180], [3376, 3575], [5672, 5871], [746...","[[490, 514], [518, 558], [558, 637], [638, 640...","[[33021, 33027], [33502, 33503], [33504, 33506...",20230612_101430_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_101430_standard_comp_to_training_D1_s...,1.3,1.3,...,"[[0.888, 0.7446, 0.7627, 0.573, 0.5938, 0.2527...","[[0.8643, 0.9487, 0.8594, 0.887, 0.2457, 0.050...","[[0.8984, 0.9756, 0.98, 0.966, 0.903, 0.9233, ...","[[0.985, 0.9976, 0.995, 0.9863, 0.991, 0.7124,...","[[0.5127, 0.671, 0.6772, 0.729, 0.4365, 0.2106...","[[0.96, 0.9683, 0.9746, 0.949, 0.8516, 0.716, ...","[[0.812, 0.632, 0.6074, 0.619, 0.2316, 0.2827,...","[[0.6343, 0.714, 0.6826, 0.702, 0.477, 0.4604,...","[10000, 20000, 30000, 40000, 50000, 60000, 700...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ..."
2,2,20230612_101430_standard_comp_to_training_D1_s...,"[[980, 1181], [3376, 3575], [5672, 5871], [746...","[[490, 514], [518, 558], [558, 637], [638, 640...","[[33137, 33147], [33665, 33666], [33668, 33669...",20230612_101430_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_101430_standard_comp_to_training_D1_s...,1.4,1.4,...,"[[0.3755, 0.5464, 0.856, 0.553, 0.0804, 0.3162...","[[0.992, 0.9385, 0.9673, 0.8193, 0.746, 0.736,...","[[0.997, 0.9976, 0.9917, 0.962, 0.9106, 0.816,...","[[0.01095, 0.264, 0.2734, 0.2175, 0.141, 0.365...","[[0.0878, 0.8496, 0.1498, 0.424, 0.4126, 0.293...","[[0.01794, 0.2399, 0.276, 0.3364, 0.2107, 0.52...","[[0.06, 0.8145, 0.1873, 0.592, 0.4753, 0.332, ...","[[0.2937, 0.4001, 0.7964, 0.9, 0.3484, 0.509, ...","[10000, 20000, 30000, 40000, 50000, 60000, 700...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ..."
3,2,20230612_101430_standard_comp_to_training_D1_s...,"[[980, 1180], [3376, 3575], [5672, 5871], [746...","[[490, 514], [518, 558], [558, 637], [638, 640...","[[33021, 33027], [33502, 33503], [33504, 33506...",20230612_101430_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_101430_standard_comp_to_training_D1_s...,1.4,1.4,...,"[[0.3755, 0.5464, 0.856, 0.553, 0.0804, 0.3162...","[[0.992, 0.9385, 0.9673, 0.8193, 0.746, 0.736,...","[[0.997, 0.9976, 0.9917, 0.962, 0.9106, 0.816,...","[[0.01095, 0.264, 0.2734, 0.2175, 0.141, 0.365...","[[0.0878, 0.8496, 0.1498, 0.424, 0.4126, 0.293...","[[0.01794, 0.2399, 0.276, 0.3364, 0.2107, 0.52...","[[0.06, 0.8145, 0.1873, 0.592, 0.4753, 0.332, ...","[[0.2937, 0.4001, 0.

In [43]:
LFP_TRACES_DF.to_pickle("./proc/rce2_spectral_coherence.pkl")

# Calculate Granger's

In [44]:
for region_1, region_2 in brain_region_pairs:
    # Define base name for pair
    region_1_base_name = region_1.split('_')[0]
    region_2_base_name = region_2.split('_')[0]

    pair_base_name = f"{region_1_base_name}_{region_2_base_name}"
    print(pair_base_name)

    try:
        # Define column names
        multitaper_col = f"{pair_base_name}_granger_multitaper"
        connectivity_col = f"{pair_base_name}_granger_connectivity"
        frequencies_col = f"{pair_base_name}_granger_frequencies"
        granger_1_2_col = f"{region_1_base_name}_{region_2_base_name}_granger_all_frequencies_all_windows"
        granger_2_1_col = f"{region_2_base_name}_{region_1_base_name}_granger_all_frequencies_all_windows"

        # Apply Multitaper function
        LFP_TRACES_DF[multitaper_col] = LFP_TRACES_DF.apply(
            lambda x: Multitaper(
                time_series=np.array([x[region_1], x[region_2]]).T, 
                sampling_frequency=RESAMPLE_RATE, 
                time_halfbandwidth_product=TIME_HALFBANDWIDTH_PRODUCT, 
                time_window_step=TIME_WINDOW_STEP, 
                time_window_duration=TIME_WINDOW_DURATION
            ), 
            axis=1
        )
    
        # Apply Connectivity function
        LFP_TRACES_DF[connectivity_col] = LFP_TRACES_DF[multitaper_col].apply(
            lambda x: Connectivity.from_multitaper(x)
        )

        # Apply frequencies and granger functions
        LFP_TRACES_DF[frequencies_col] = LFP_TRACES_DF[connectivity_col].apply(
            lambda x: x.frequencies
        )
        
        LFP_TRACES_DF[granger_1_2_col] = LFP_TRACES_DF[connectivity_col].apply(
            lambda x: x.pairwise_spectral_granger_prediction()[:,:,0,1]
        )

        LFP_TRACES_DF[granger_2_1_col] = LFP_TRACES_DF[connectivity_col].apply(
            lambda x: x.pairwise_spectral_granger_prediction()[:,:,1,0]
        )

        LFP_TRACES_DF[granger_1_2_col] = LFP_TRACES_DF[granger_1_2_col].apply(lambda x: x.astype(np.float16))
        LFP_TRACES_DF[granger_2_1_col] = LFP_TRACES_DF[granger_2_1_col].apply(lambda x: x.astype(np.float16))
        
    except Exception as e: 
        print(e)

    # Drop temporary columns
    LFP_TRACES_DF = LFP_TRACES_DF.drop(columns=[multitaper_col, connectivity_col], errors="ignore")

BLA_LH


Maximum iterations reached. 6706 of 6828 converged
Maximum iterations reached. 6706 of 6828 converged
Maximum iterations reached. 6819 of 6828 converged
Maximum iterations reached. 6819 of 6828 converged
Maximum iterations reached. 6779 of 6833 converged
Maximum iterations reached. 6779 of 6833 converged
Maximum iterations reached. 6819 of 6833 converged
Maximum iterations reached. 6819 of 6833 converged
Maximum iterations reached. 6783 of 6840 converged
Maximum iterations reached. 6783 of 6840 converged
Maximum iterations reached. 6839 of 6841 converged
Maximum iterations reached. 6839 of 6841 converged
Maximum iterations reached. 6767 of 6804 converged
Maximum iterations reached. 6767 of 6804 converged
Maximum iterations reached. 6767 of 6804 converged
Maximum iterations reached. 6346 of 6678 converged
Maximum iterations reached. 6346 of 6678 converged
Maximum iterations reached. 6346 of 6678 converged
Maximum iterations reached. 6703 of 6746 converged
Maximum iterations reached. 670

BLA_MD


Maximum iterations reached. 6718 of 6828 converged
Maximum iterations reached. 6718 of 6828 converged
Maximum iterations reached. 6819 of 6828 converged
Maximum iterations reached. 6819 of 6828 converged
Maximum iterations reached. 6743 of 6833 converged
Maximum iterations reached. 6743 of 6833 converged
Maximum iterations reached. 6801 of 6833 converged
Maximum iterations reached. 6801 of 6833 converged
Maximum iterations reached. 6730 of 6840 converged
Maximum iterations reached. 6730 of 6840 converged
Maximum iterations reached. 6837 of 6841 converged
Maximum iterations reached. 6837 of 6841 converged
Maximum iterations reached. 6717 of 6804 converged
Maximum iterations reached. 6717 of 6804 converged
Maximum iterations reached. 6717 of 6804 converged
Maximum iterations reached. 6331 of 6678 converged
Maximum iterations reached. 6331 of 6678 converged
Maximum iterations reached. 6331 of 6678 converged
Maximum iterations reached. 6684 of 6746 converged
Maximum iterations reached. 668

BLA_mPFC


...

Maximum iterations reached. 6548 of 6828 converged
Maximum iterations reached. 6548 of 6828 converged
Maximum iterations reached. 6815 of 6828 converged
Maximum iterations reached. 6815 of 6828 converged
Maximum iterations reached. 6801 of 6833 converged
Maximum iterations reached. 6801 of 6833 converged
Maximum iterations reached. 6784 of 6833 converged
Maximum iterations reached. 6784 of 6833 converged
Maximum iterations reached. 6814 of 6840 converged
Maximum iterations reached. 6814 of 6840 converged
Maximum iterations reached. 6840 of 6841 converged
Maximum iterations reached. 6840 of 6841 converged
Maximum iterations reached. 6774 of 6804 converged
Maximum iterations reached. 6774 of 6804 converged
Maximum iterations reached. 6774 of 6804 converged
Maximum iterations reached. 6307 of 6678 converged
Maximum iterations reached. 6307 of 6678 converged
Maximum iterations reached. 6307 of 6678 converged
Maximum iterations reached. 6680 of 6746 converged
Maximum iterations reached. 668

MD_vHPC


Maximum iterations reached. 6540 of 6828 converged
Maximum iterations reached. 6540 of 6828 converged
Maximum iterations reached. 6782 of 6828 converged
Maximum iterations reached. 6782 of 6828 converged
Maximum iterations reached. 6825 of 6833 converged
Maximum iterations reached. 6825 of 6833 converged
Maximum iterations reached. 6808 of 6833 converged
Maximum iterations reached. 6808 of 6833 converged
Maximum iterations reached. 6832 of 6840 converged
Maximum iterations reached. 6832 of 6840 converged
Maximum iterations reached. 6830 of 6841 converged
Maximum iterations reached. 6830 of 6841 converged
Maximum iterations reached. 6795 of 6804 converged
Maximum iterations reached. 6795 of 6804 converged
Maximum iterations reached. 6795 of 6804 converged
Maximum iterations reached. 6356 of 6678 converged
Maximum iterations reached. 6356 of 6678 converged
Maximum iterations reached. 6356 of 6678 converged
Maximum iterations reached. 6722 of 6746 converged
Maximum iterations reached. 672

mPFC_vHPC


Maximum iterations reached. 6568 of 6828 converged
Maximum iterations reached. 6568 of 6828 converged
Maximum iterations reached. 6827 of 6828 converged
Maximum iterations reached. 6827 of 6828 converged
Maximum iterations reached. 6827 of 6833 converged
Maximum iterations reached. 6827 of 6833 converged
Maximum iterations reached. 6810 of 6833 converged
Maximum iterations reached. 6810 of 6833 converged
Maximum iterations reached. 6833 of 6840 converged
Maximum iterations reached. 6833 of 6840 converged
Maximum iterations reached. 6839 of 6841 converged
Maximum iterations reached. 6839 of 6841 converged
Maximum iterations reached. 6795 of 6804 converged
Maximum iterations reached. 6795 of 6804 converged
Maximum iterations reached. 6795 of 6804 converged
Maximum iterations reached. 6338 of 6678 converged
Maximum iterations reached. 6338 of 6678 converged
Maximum iterations reached. 6338 of 6678 converged
Maximum iterations reached. 6718 of 6746 converged
Maximum iterations reached. 671

- Getting the timestamps of the granger

In [45]:
LFP_TRACES_DF["granger_timestamps"] = LFP_TRACES_DF["lfp_timestamps"].apply(lambda x: x[(RESAMPLE_RATE//2):(-RESAMPLE_RATE//2):(RESAMPLE_RATE//2)])


- Making sure that the timestamps for granger makes sense with shape and values

In [46]:
LFP_TRACES_DF["granger_timestamps"].head().apply(lambda x: x.shape)

0    (6828,)
1    (6828,)
2    (6828,)
3    (6828,)
4    (6833,)
Name: granger_timestamps, dtype: object

In [47]:
LFP_TRACES_DF[[col for col in LFP_TRACES_DF.columns if "granger_all_frequencies_all_windows" in col][0]].iloc[0].shape

(6828, 500)

In [48]:
LFP_TRACES_DF[[col for col in LFP_TRACES_DF.columns if "lfp_timestamps" in col][0]].iloc[0]

array([       0,       20,       40, ..., 68293300, 68293320, 68293340],
      dtype=int32)

In [49]:
LFP_TRACES_DF[[col for col in LFP_TRACES_DF.columns if "granger_timestamps" in col][0]].iloc[0]

array([   10000,    20000,    30000, ..., 68260000, 68270000, 68280000],
      dtype=int32)

- Checking if the right frequencies are being used

In [50]:
LFP_TRACES_DF[[col for col in LFP_TRACES_DF.columns if "granger_frequencies" in col]].head()

,BLA_LH_granger_frequencies,BLA_MD_granger_frequencies,BLA_mPFC_granger_frequencies,BLA_vHPC_granger_frequencies,LH_MD_granger_frequencies,LH_mPFC_granger_frequencies,LH_vHPC_granger_frequencies,MD_mPFC_granger_frequencies,MD_vHPC_granger_frequencies,mPFC_vHPC_granger_frequencies
0,"[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ..."
1,"[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ..."
2,"[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ..."
3,"[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ..."
4,"[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ..."


In [51]:
LFP_TRACES_DF["granger_calculation_frequencies"] = LFP_TRACES_DF[[col for col in LFP_TRACES_DF.columns if "granger_frequencies" in col][0]].copy()

- Dropping unnecessary columns

In [52]:
LFP_TRACES_DF = LFP_TRACES_DF.drop(columns=[col for col in LFP_TRACES_DF.columns if "granger_frequencies" in col], errors="ignore")

In [53]:
LFP_TRACES_DF.head()

,cohort,session_dir,tone_frames,box_1_port_entry_frames,box_2_port_entry_frames,video_name,session_path,recording,current_subject,subject,...,LH_vHPC_granger_all_frequencies_all_windows,vHPC_LH_granger_all_frequencies_all_windows,MD_mPFC_granger_all_frequencies_all_windows,mPFC_MD_granger_all_frequencies_all_windows,MD_vHPC_granger_all_frequencies_all_windows,vHPC_MD_granger_all_frequencies_all_windows,mPFC_vHPC_granger_all_frequencies_all_windows,vHPC_mPFC_granger_all_frequencies_all_windows,granger_timestamps,granger_calculation_frequencies
0,2,20230612_101430_standard_comp_to_training_D1_s...,"[[980, 1181], [3376, 3575], [5672, 5871], [746...","[[490, 514], [518, 558], [558, 637], [638, 640...","[[33137, 33147], [33665, 33666], [33668, 33669...",20230612_101430_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_101430_standard_comp_to_training_D1_s...,1.3,1.3,...,"[[1.212, 0.6343, 0.2808, 0.6133, 0.635, 1.502,...","[[0.3904, 0.2316, 0.02307, 0.1547, 0.3137, 0.7...","[[1.767, 1.53, 1.763, 0.898, 1.155, 1.547, 0.1...","[[1.765, 0.7905, 1.229, 0.71, 0.3438, 0.6763, ...","[[1.93, 0.4773, 0.3132, 0.615, 0.8076, 0.6006,...","[[0.1791, 0.1605, 0.01888, 0.0991, 0.268, 0.35...","[[0.3865, 0.2566, 0.1313, 0.1837, 0.2905, nan,...","[[0.000577, 0.1996, 0.2228, 0.5625, 0.6416, 0....","[10000, 20000, 30000, 40000, 50000, 60000, 700...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ..."
1,2,20230612_101430_standard_comp_to_training_D1_s...,"[[980, 1180], [3376, 3575], [5672, 5871], [746...","[[490, 514], [518, 558], [558, 637], [638, 640...","[[33021, 33027], [33502, 33503], [33504, 33506...",20230612_101430_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_101430_standard_comp_to_training_D1_s...,1.3,1.3,...,"[[1.212, 0.6343, 0.2808, 0.6133, 0.635, 1.502,...","[[0.3904, 0.2316, 0.02307, 0.1547, 0.3137, 0.7...","[[1.767, 1.53, 1.763, 0.898, 1.155, 1.547, 0.1...","[[1.765, 0.7905, 1.229, 0.71, 0.3438, 0.6763, ...","[[1.93, 0.4773, 0.3132, 0.615, 0.8076, 0.6006,...","[[0.1791, 0.1605, 0.01888, 0.0991, 0.268, 0.35...","[[0.3865, 0.2566, 0.1313, 0.1837, 0.2905, nan,...","[[0.000577, 0.1996, 0.2228, 0.5625, 0.6416, 0....","[10000, 20000, 30000, 40000, 50000, 60000, 700...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ..."
2,2,20230612_101430_standard_comp_to_training_D1_s...,"[[980, 1181], [3376, 3575], [5672, 5871], [746...","[[490, 514], [518, 558], [558, 637], [638, 640...","[[33137, 33147], [33665, 33666], [33668, 33669...",20230612_101430_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_101430_standard_comp_to_training_D1_s...,1.4,1.4,...,"[[2.48, 1.336, 0.662, 0.2261, 0.1403, 0.1016, ...","[[1.638, 0.751, 0.1073, 0.3499, 0.261, 0.593, ...","[[0.11383, 0.06714, 0.1582, 0.02718, 0.1484, 0...","[[1.162, 0.1947, 0.391, 0.1361, 0.209, 0.6934,...","[[1.679, 1.423, 0.822, 0.2617, 0.0925, 0.03287...","[[0.985, 0.4849, 0.1306, 0.3406, 0.2705, 0.308...","[[1.434, 2.254, 0.567, 0.3025, 0.1377, 0.6514,...","[[nan, 0.83, 1.557, 0.672, 0.3513, 0.01386, 0....","[10000, 20000, 30000, 40000, 50000, 60000, 700...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ..."
3,2,20230612_101430_standard_comp_to_training_D1_s...,"[[980, 1180], [3376, 3575], [5672, 5871], [746...","[[490, 514], [518, 558], [558, 637], [638, 640...","[[33021, 33027], [33502, 33503], [33504, 33506...",20230612_101430_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_101430_standard_comp_to_training_D1_s...,1.4,1.4,...,"[[2.48, 1.336, 0.662, 0.2261, 0.1403, 0.1016, ...","[[1.638, 0.751, 0.1073, 0.3499, 0.261, 0.593, ...","[[0.11383, 0.06714, 0.1582, 0.02718, 0.1484, 0...","[[1.162, 0.1947, 0.391, 0.1361, 0.209, 0.6934,...","[[1.679, 1.423, 0.822, 0.2617, 0.0925, 0.03287...","[[0.985, 0.4849, 0.1306, 0.3406, 0.2705, 0.308...","[[1.434, 2.254, 0.567, 0.3025, 0.1377, 0.6514,...","[[nan, 0.83, 1.557, 0.672, 0.3513, 0.0

## Calculating the averages

In [54]:
LFP_TRACES_DF.columns

Index(['cohort', 'session_dir', 'tone_frames', 'box_1_port_entry_frames',
       'box_2_port_entry_frames', 'video_name', 'session_path', 'recording',
       'current_subject', 'subject', 'all_subjects', 'first_timestamp',
       'last_timestamp', 'video_timestamps', 'tone_timestamps',
       'box_1_port_entry_timestamps', 'box_2_port_entry_timestamps',
       'lfp_timestamps', 'mPFC_lfp_trace', 'MD_lfp_trace', 'LH_lfp_trace',
       'BLA_lfp_trace', 'vHPC_lfp_trace',
       'mPFC_power_all_frequencies_all_windows',
       'MD_power_all_frequencies_all_windows',
       'LH_power_all_frequencies_all_windows',
       'BLA_power_all_frequencies_all_windows',
       'vHPC_power_all_frequencies_all_windows', 'power_timestamps',
       'power_calculation_frequencies',
       'BLA_LH_coherence_all_frequencies_all_windows',
       'BLA_MD_coherence_all_frequencies_all_windows',
       'BLA_mPFC_coherence_all_frequencies_all_windows',
       'BLA_vHPC_coherence_all_frequencies_all_windows',
   

In [55]:
# LFP_TRACES_DF.to_pickle("./proc/rce2_spectral_granger.pkl")
LFP_TRACES_DF.to_pickle("./proc/rce_pilot_2_02_standard_spectral.pkl")
# LFP_TRACES_DF.to_pickle("/blue/npadillacoreano/ryoi360/projects/reward_comp/final_proc/rce_pilot_2_02_spectral_granger.pkl")

In [56]:
raise ValueError()

ValueError: 